## Sobre la base de datos

Está relacionada al trabajo:
[PepLab Platform: Database and Software Tools for Analysis of Food-Derived Bioactive Peptides](https://www.mdpi.com/2076-3417/13/2/961)

Contiene registros positivos de antifúngicos.

Usaremos la biblioteca `Biopython` que es muy útil para trabajar con secuencias biológicas.

In [1]:
%pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.9 MB/s eta 0:00:00


In [2]:
from Bio import SeqIO
import pandas as pd

In [3]:
fasta = '/content/PepLab_Antifungal.fasta'

## Secuencias positivas

In [4]:
data = []
try:
    for record in SeqIO.parse(fasta, "fasta"):
        data.append({
            "ID": record.id,
            "Description": record.description,
            "Sequence": str(record.seq)
        })

    df = pd.DataFrame(data)
    display(df.head())

except FileNotFoundError:
    print(f"Error: El archivo no se encuentra en la ruta especificada: {fasta}")
except Exception as e:
    print(f"Ocurrió un error al parsear el archivo fasta: {e}")

,ID,Description,Sequence
0,PL18,PL18,SYTFH
1,PL52,PL52,YIACYNGATSYNQKFK
2,PL114,PL114,QHIRELTRSE
3,PL400,PL400,DASNRAT
4,PL472,PL472,LVSNLES


In [6]:
df_afp = df[['Sequence']]

# Agregar una columna de label (si tiene actividad antifúngica o no)
df_afp['label'] = 1

/tmp/ipython-input-46429444.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_afp['label'] = 1


In [7]:
df_afp.shape

(39, 2)

## Verificar duplicados

In [8]:
df_afp["Sequence"].unique().shape

(39,)

In [9]:
df_afp = df_afp.drop_duplicates().copy()

In [10]:
df_afp.shape

(39, 2)

## Verificar canónicos

In [11]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [12]:
# Verificar cuáles secuencias válidas
df_afp["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df_afp["Sequence"]]

In [13]:
no_validas = df_afp[~df_afp["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 0


In [14]:
df_validas = df_afp[df_afp["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 39


In [15]:
display(df_validas)

,Sequence,label
0,SYTFH,1
1,YIACYNGATSYNQKFK,1
2,QHIRELTRSE,1
3,DASNRAT,1
4,LVSNLES,1
5,PNPLKAM,1
6,HPLKQYWWRPSI,1
7,GILGKIWEGVKSLI,1
8,ARQGVRGGAMD,1
9,WKLFKKILKVL,1


In [16]:
df_validas = df_validas.rename(columns={'Sequence': 'sequence'})
display(df_validas.head())

,sequence,label
0,SYTFH,1
1,YIACYNGATSYNQKFK,1
2,QHIRELTRSE,1
3,DASNRAT,1
4,LVSNLES,1


In [17]:
df_validas.to_csv('PepLab_labeled.csv', index=False, header=True)

## Obtener metadatos

In [18]:
import json

In [19]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [20]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [21]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [24]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "Pep-Lab_db")

In [25]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df_validas)

In [26]:
export_json("metadata.json", dict_metadata)